In [1]:
# Importing the required Libraries
import pandas as pd
import sqlite3

In [3]:
# Load orders.csv
orders_df = pd.read_csv(r"C:\Users\Kashish\Downloads\orders.csv")

In [4]:
# Load users.json
users_df = pd.read_json(r"C:\Users\Kashish\Downloads\users.json")

In [6]:
# Load restaurants.sql
conn = sqlite3.connect("restaurants.db")

with open(r"C:\Users\Kashish\Downloads\restaurants.sql", "r") as file:
    sql_script = file.read()
conn.executescript(sql_script)
restaurants_df = pd.read_sql(
    "SELECT * FROM restaurants", conn
)

In [7]:
# Merge the Data(left joins)
orders_users_df = pd.merge(
    orders_df,
    users_df,
    on = "user_id",
    how = "left"
)
final_df = pd.merge(
    orders_users_df,
    restaurants_df,
    on = "restaurant_id",
    how = "left"
)

In [8]:
# Save Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index = False)

In [18]:
# Which city has the highest total revenue (total_amount) from Gold members?  
gold_df = final_df[final_df["membership"] == "Gold"]
city_gold_revenue = (
    gold_df
    .groupby("city")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by = "total_amount", ascending = False)
)
top_city = city_gold_revenue.iloc[0]
print("City with highest total revenue from gold members: ", top_city["city"])

City with highest total revenue from gold members:  Chennai


In [15]:
# Which cuisine has the highest average order value across all orders?
cuisine_avg_order = (
    final_df.groupby("cuisine")["total_amount"]
    .mean()
    .reset_index()
    .sort_values(by = "total_amount", ascending = False)
)
top_cuisine = cuisine_avg_order.iloc[0]
print("Cuisine with highest average order value: ", top_cuisine["cuisine"])

Cuisine with highest average order value:  Mexican


In [16]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_total_orders = (
    final_df.groupby("user_id")["total_amount"]
    .sum()
    .reset_index()
)
distinct_users_count = user_total_orders[user_total_orders["total_amount"] > 1000]["user_id"].nunique()
print("Number of distinct users are: ", distinct_users_count)

Number of distinct users are:  2544


In [17]:
# Which restaurant rating range generated the highest total revenue?
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins = rating_bins,
    labels = rating_labels,
    include_lowest = True
)
rating_revenue = (
    final_df.groupby("rating_range")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by = "total_amount", ascending = False)
)
top_range = rating_revenue.iloc[0]
print("Restaurant rating range that generated highest revenue is: ", top_range["rating_range"])

Restaurant rating range that generated highest revenue is:  4.6-5.0


C:\Users\Kashish\AppData\Local\Temp\ipykernel_22464\2043116195.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"]


In [19]:
# Among Gold members, which city has the highest average order value?
city_avg_order = (
    gold_df.groupby("city")["total_amount"]
    .mean()
    .reset_index()
    .sort_values(by = "total_amount", ascending = False)
)
top_city = city_avg_order.iloc[0]
print("City with highest average order value by gold members: ", top_city["city"])

City with highest average order value by gold members:  Chennai


In [21]:
# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
cuisine_stats = (
    final_df.groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .reset_index()
)

# Sort by fewest restaurants first, then highest revenue
cuisine_stats_sorted = cuisine_stats.sort_values(
    by=["distinct_restaurants", "total_revenue"],
    ascending=[True, False]
)

# Top cuisine based on the condition
top_cuisine = cuisine_stats_sorted.iloc[0]

print("Cuisine with lowest number of distinct restaurants but still contributes significant revenue: ",top_cuisine["cuisine"])


Cuisine with lowest number of distinct restaurants but still contributes significant revenue:  Chinese


In [22]:
# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])

percentage_gold_orders = round((gold_orders / total_orders) * 100)

print("Percentage of total orders placed by gold members: ",percentage_gold_orders)


Percentage of total orders placed by gold members:  50


In [29]:
# Which restaurant has the highest average order value but less than 20 total orders?
restaurant_stats = (
    final_df.groupby("restaurant_name_x")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
    .reset_index()
)

filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

top_restaurant = filtered_restaurants.sort_values(
    by="avg_order_value", ascending=False
).iloc[0]

print("Restaurant with highest average order value but less than 20 total orders: ",top_restaurant["restaurant_name_x"])


Restaurant with highest average order value but less than 20 total orders:  Hotel Dhaba Multicuisine


In [32]:
# Which combination contributes the highest revenue?
combo_revenue = (
    final_df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
)

filtered_combos = combo_revenue[
    (
        (combo_revenue["membership"] == "Gold") & (combo_revenue["cuisine"] == "Indian")
    ) |
    (
        (combo_revenue["membership"] == "Gold") & (combo_revenue["cuisine"] == "Italian")
    ) |
    (
        (combo_revenue["membership"] == "Regular") & (combo_revenue["cuisine"] == "Indian")
    ) |
    (
        (combo_revenue["membership"] == "Regular") & (combo_revenue["cuisine"] == "Chinese")
    )
]

top_combo = filtered_combos.sort_values(
    by="total_amount", ascending=False
).iloc[0]

print(top_combo["membership"], "+", top_combo["cuisine"])


Gold + Italian


In [33]:
# During which quarter of the year is the total revenue highest?
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

quarter_revenue = (
    final_df.groupby("quarter")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

top_quarter = quarter_revenue.iloc[0]

print(top_quarter["quarter"])


2023Q3


C:\Users\Kashish\AppData\Local\Temp\ipykernel_22464\1577386766.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


In [34]:
# How many total orders were placed by users with Gold membership?
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]

print(gold_orders_count)

4987


In [35]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()

print(round(hyderabad_revenue))

1889367


In [36]:
# How many distinct users placed at least one order?
distinct_users = final_df["user_id"].nunique()

print(distinct_users)

2883


In [37]:
# What is the average order value (rounded to 2 decimals) for Gold members?
avg_order_value_gold = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()

print(round(avg_order_value_gold, 2))

797.15


In [38]:
# How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]

print(high_rating_orders)

3374


In [40]:
# How many orders were placed in the top revenue city among Gold members only?
top_city = city_gold_revenue.iloc[0]["city"]

orders_count = gold_df[gold_df["city"] == top_city].shape[0]

print(orders_count)

1337


In [41]:
print(len(final_df))

10000
